# Advanced options for the DelftaCalculator

This tutorials looks a bit more in-depth at the different options you can pass to the DelftaCalculator. You can refer to the basic tutorial ([delta_vs_direct.ipynb](delta_vs_direct.ipynb)) to get started with the default settings. Again, we'll start with imports: 

In [1]:
from openbabel.pybel import readstring
import pandas as pd
from delfta.calculator import DelftaCalculator

The options for the calculator (with their respective defaults) are `tasks=None`, `delta=True`, `force3d=True`, `addh=True`, `xtbopt=False`, `verbose=True`, and `progress=True`, `return_optmols=False`, and `models=None`. `verbose` and `progress` just modify how much you see during the computation, but not the computation itself. Let's look at the other options in detail.

## Tasks

This defines which properties the calculator should predict. You can either pass a list with any combination of the following keys, or simply leave the default (`None`) to get all the values. 

| Property                                      | Key         | Unit  |
|-----------------------------------------------|-------------|-------|
| Formation energy                              | `"E_form"`  | $E_h$ |
| Energy of highest occupied molecular orbital  | `"E_homo"`  | $E_h$ |
| Energy of lowest unoccupied molecular orbital | `"E_lumo"`  | $E_h$ |
| HOMO-LUMO gap                                 | `"E_gap"`   | $E_h$ |
| Molecular dipole                              | `"dipole"`  | D     |
| Mulliken partial charges                      | `"charges"` | $e$   |
| Wiberg bond orders                            | `"wbo"`     | -     |

Note that xTB needs to be run only once for all of them (if `delta=True`, see later), and that HOMO/LUMO/gap energies and the dipole are predicted in a multi-task setting (all via the same network), so the computational cost does not scale linearly with the number of requested properties. 



## delta

This defines whether or not to use the $\Delta$-prediction approach, *i.e.*, whether to compute the requested values with the semi-empirical GFN2-xTB method, and use the network to predict a correction to this value to obtain an approximation of the DFT value ($\omega$B97X-D/def2-SVP). This is the default (`delta=True`), but you can set it to `False` to directly predict the requested properties from the molecular structure. This removes the need to compute xTB and thus speeds up the process a little bit (though this only makes a noticable differences when you run the calculator for large numbers of molecules).

## force3d

This defines whether you want to use the Merck Molecular Force Field 94 (MMFF94) as implemented in Openbabel to create 3D coordinates for molecules that don't have them. All the quantum mechanical properties that `DelftaCalculator` provides depend on the molecular geometry, so you really shouldn't be passing a 2D molecule and expect reasonable results (of course flat structures like benzene are fine). This defaults to `force3d=True`, and will not affect any molecules you pass that already have a 3D geometry. 

## addh

This defines whether you want to add hydrogens to the molecule. If enabled, we're using Openbabel to check if there's hydrogens missing, and add them accordingly. Just as with `force3d`, it's important to include hydrogens in the molecule rather than using only the heavy atoms in the quantum mechanical calculations/predictions. Note that hydrogens are often omited in SMILES notation. This option also defaults to `addh=True` and won't affect any molecules that already have explicit hydrogens added. 

## xtbopt & return_optmols

This option lets you use GFN2-xTB to optimize the 3D structure of a molecule. This can be useful if you created the conformation with a force field (or used `force3d` to do this), but want to optimize the structure a bit more thoroughly with a more precise method. Let's run the calculator twice, once generating coordinates using the MMFF94 force field, and once adding the GFN2-xTB geometry optimization to the pipeline as well.

In [2]:
smiles = "O=C(C)Oc1ccccc1C(=O)O" # aspirin
mol = readstring("smi", smiles)
calc_delta = DelftaCalculator(delta=True, xtbopt=False, return_optmols=True) 
predictions_delta_mmff94, opt_mols_mmff94 = calc_delta.predict(mol)
opt_mol_mmff94 = opt_mols_mmff94[0]

calc_delta = DelftaCalculator(delta=True, xtbopt=True, return_optmols=True) 
predictions_delta_xtb, opt_mols_xtb = calc_delta.predict(mol)
opt_mol_xtb = opt_mols_xtb[0]

2021/08/17 11:14:36 AM | DelFTa | INFO: Assigned MMFF94 coordinates and added hydrogens to molecules at position(s) [0]
2021/08/17 11:14:36 AM | DelFTa | INFO: Now running xTB...
100%|██████████| 1/1 [00:00<00:00, 24.79it/s]
2021/08/17 11:14:36 AM | DelFTa | INFO: Now running network for model single_energy_delta...
100%|██████████| 1/1 [00:00<00:00,  9.11it/s]
2021/08/17 11:14:38 AM | DelFTa | INFO: Now running network for model charges_delta...
100%|██████████| 1/1 [00:00<00:00, 158.51it/s]
2021/08/17 11:14:38 AM | DelFTa | INFO: Now running network for model multitask_delta...
100%|██████████| 1/1 [00:00<00:00, 144.89it/s]
2021/08/17 11:14:38 AM | DelFTa | INFO: Now running network for model wbo_delta...
100%|██████████| 1/1 [00:00<00:00, 155.81it/s]
2021/08/17 11:14:38 AM | DelFTa | INFO: Now running xTB...
100%|██████████| 1/1 [00:00<00:00,  2.26it/s]
2021/08/17 11:14:38 AM | DelFTa | INFO: Now running network for model single_energy_delta...
100%|██████████| 1/1 [00:00<00:00, 138

Next, we'll visualize both outputs (interactive, so feel free to move around the molecules). As you can see, there are some small differences between the results of both methods, and for more complicated or flexible molecules, there's a chance those will be larger. Note that the interactive visualizations may not be available online - simply download a version of the notebook to your machine to try it out. 

In [3]:
import py3Dmol
size=(1000,500)
view = py3Dmol.view(width=size[0], height=size[1], linked=True, viewergrid=(1,2))
view.removeAllModels()
view.addModel(opt_mol_mmff94.write("xyz"), "xyz", viewer=(0,0))
view.addModel(opt_mol_xtb.write("xyz"), "xyz", viewer=(0,1))
view.addLabel("MMFF94", {"position":{"x":3,"y":-3,"z":0.0}}, viewer=(0,0))
view.addLabel("GFN2-xTB", {"position":{"x":3,"y":-3,"z":0.0}}, viewer=(0,1))
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Let's also take a brief look at the resulting predictions for both geometries:

In [4]:
res_mmff94 = {key: val[0] for key, val in predictions_delta_mmff94.items() if key not in ["charges", "wbo"]}
res_xtb = {key: val[0] for key, val in predictions_delta_xtb.items() if key not in ["charges", "wbo"]}
pd.DataFrame.from_dict({"MMFF94": res_mmff94, "GFN2-xTB": res_xtb})


,MMFF94,GFN2-xTB
E_form,-3.869898,-3.895653
E_homo,-0.345097,-0.343340
E_lumo,-0.011147,0.000587
E_gap,0.333909,0.343828
dipole,3.698956,3.716143


## models

Here you can pass a list of paths to specific model checkpoints, if you want to use those instead of the default production models. The most straight-forward use case for this would be if you want to try out models trained on different training set sizes (which you can download from the link in the README.) Alternatively, if you trained your own models, you can pass those here as well. You can only set `tasks` or `models` manually, but not both, and if you set `models`, the corresponding `tasks` will be infered from the model names. Also make sure that you're passing the correct `delta` argument for the models you're using. Unless you modify the file yourself (`delfta/models/norm.pt`), the normalization values for the entire dataset will be used. Normalization values for the training set only are available together with the models for different training set sizes. 